<a href="https://colab.research.google.com/github/bishalbar77/Amazon_ML/blob/main/Amazon_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Code Author**

*   Bishal Rana

In [ ]:
mkdir amazon_contest

In [ ]:
cd amazon_contest/

/content/amazon_contest


In [3]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip

--2021-07-31 19:29:27--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.164.26
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.164.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip’

dataset52a7b21.zip  100%[===================>]   1012M  14.6MB/s    in 72s     

2021-07-31 19:30:39 (14.1 MB/s) - ‘dataset52a7b21.zip’ saved [1061576029/1061576029]



In [ ]:
!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
from nltk.corpus import stopwords

In [ ]:
trainData = pd.read_csv('/content/amazon_contest/content/dataset/train.csv',escapechar = "\\" , quoting=3)

In [ ]:
trainData

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4
...,...,...,...,...,...
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933
2903022,Type-C to 3.5 MM for Oppo R17 Pro Type-C to 3....,Still want to use your favorite earphones/head...,"[Indian Connectors: Made for Indian sockets, t...",SHOPBELL,14790


In [ ]:
testData = pd.read_csv('/content/amazon_contest/content/dataset/test.csv',escapechar = "\\" , quoting=3)

In [ ]:
testData

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero
...,...,...,...,...,...
110770,110771,AAHNA E MALL OneBlade Hybrid Trimmer Shaver An...,<p>1-All In One Hyper Advanced Smart Rechargea...,"[Unique One Blade can style, trim and shave, w...",Generic
110771,110772,Grin Health N99 Anti Pollution Reusable Washab...,"<p>SIZE GUIDE : M - (35- 65 Kg), L - (49- 72 K...",[PROTECTION: Filtration rate up to ≥99 percent...,Grin Health
110772,110773,Asian Army Pink Ultra reusable respirator clot...,Asian HyperProtect A95 masks have been enginee...,[Reusable and environment friendly: These mask...,ASIAN
110773,110774,IM Safe 3 Ply Non-Woven Disposable Surgical Fa...,This 3 Ply Disposable face mask is manufacture...,[3 Ply Mask: Genuine 3 Ply Mask. 25 GSM Spun B...,Intermarket


In [ ]:
sample_submission = pd.read_csv('/content/dataset/sample_submission.csv',escapechar = "\\" , quoting=3)

In [ ]:
sample_submission

,PRODUCT_ID,BROWSE_NODE_ID
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4


In [ ]:
trainData.isna().sum()

TITLE                 71
DESCRIPTION       723664
BULLET_POINTS     166263
BRAND              56737
BROWSE_NODE_ID         0
dtype: int64

In [ ]:
testData.isna().sum()

PRODUCT_ID           0
TITLE                7
DESCRIPTION      10469
BULLET_POINTS     6786
BRAND             2430
dtype: int64

In [ ]:
trainData.dropna(inplace=True)

In [ ]:
trainData.shape

(2110586, 5)

# Data Cleaning

**Removing HTML tags**

In [ ]:
def removeHTML(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

**Removing Punctuations**

In [ ]:
def removePunctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

**Removing Stopwords**

In [ ]:
stopwards = set(stopwords.words('english'))
def removeStopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwards])


In [ ]:
def lowerCasing(text):
  return text.lower()

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
stemmer = PorterStemmer()
def stemWords(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [ ]:
trainData.columns,testData.columns

(Index(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND', 'BROWSE_NODE_ID'], dtype='object'),
 Index(['PRODUCT_ID', 'TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND'], dtype='object'))

In [ ]:
def processAll(col):
  col = col.apply(lambda text: removeHTML(text))
  col = col.apply(lambda text: removePunctuation(text))
  col = col.apply(lambda text: removeStopwords(text))
  col = col.apply(lambda text: lowerCasing(text))
  col = col.apply(lambda text: stemWords(text))
  return col

In [ ]:
def preProcessingAllColumns(df):
  df['TITLE']=processAll(df['TITLE'])
  df['DESCRIPTION']=processAll(df['DESCRIPTION'])
  df['BULLET_POINTS']=processAll(df['BULLET_POINTS'])
  return df

In [ ]:
preProcessingAllColumns(trainData)

In [ ]:
trainData

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,pete the cat bedtime blues doll 145 inch,pete cat coolest popular cat town the new pete...,pete cat bedtime blues plush dollbased popular...,MerryMakers,0
1,the new yorker nyhm014 refrigerator magnet 2 x 35,the new yorker handsome cello wrapped hard mag...,cat in a tea cup new yorker cover artist gurbu...,The New Yorker,1
5,mens full sleeve raglan tshirts denim tshirt m...,mens full sleeve raglan tshirts denim tshirt m...,color bluesleeve full sleevematerial cottonnec...,Bhavya Enterprise,5
6,glance womens wallet black lw21,this black wallet glance treasured addition ac...,the most comfortable womens wallet that you wo...,Glance,6
7,wild animals hungry brain educational flash ca...,wild animals animals mostly stays forest requi...,playful learning flash cards develops love lea...,hungry brain,7
...,...,...,...,...,...
2903018,the nxt lvl liquid silicone soft back cover ca...,product color may slightly vary due photograph...,liquid silicone case compatible apple pro made...,Generic,794481
2903019,premium aviator sunglasses hd polarized bright...,these premium aviator sunglasses 5 color optio...,frame size lens height 56mm lens width 58 mm n...,Generic,1040
2903020,social distance stickers set 5 sticker slip re...,set 5 prints social distancing sticker self ad...,covid19 safety sticker set 5 maintain crowd co...,Generic,15199
2903021,torrto face shield pack of 5 adjustable elasti...,complete face protection torrto face shield ef...,350 microns pack of 5 pcscomplete face protect...,TORR-TO,1044933


In [ ]:
trainData.reset_index(inplace=True)

In [ ]:
trainData.head()['TITLE']

0             pete the cat bedtime blues doll 145 inch
1    the new yorker nyhm014 refrigerator magnet 2 x 35
2    mens full sleeve raglan tshirts denim tshirt m...
3                      glance womens wallet black lw21
4    wild animals hungry brain educational flash ca...
Name: TITLE, dtype: object

In [ ]:
y_train = trainData['BROWSE_NODE_ID']

In [ ]:
X_train = trainData.drop('BROWSE_NODE_ID', axis = 1)

In [ ]:
X_train, y_train

(                                                     TITLE  ...              BRAND
 0                 pete the cat bedtime blues doll 145 inch  ...        MerryMakers
 1        the new yorker nyhm014 refrigerator magnet 2 x 35  ...     The New Yorker
 5        mens full sleeve raglan tshirts denim tshirt m...  ...  Bhavya Enterprise
 6                          glance womens wallet black lw21  ...             Glance
 7        wild animals hungry brain educational flash ca...  ...       hungry brain
 ...                                                    ...  ...                ...
 2903018  the nxt lvl liquid silicone soft back cover ca...  ...            Generic
 2903019  premium aviator sunglasses hd polarized bright...  ...            Generic
 2903020  social distance stickers set 5 sticker slip re...  ...            Generic
 2903021  torrto face shield pack of 5 adjustable elasti...  ...            TORR-TO
 2903022  typec 35 mm oppo r17 pro typec 35 mm audio jac...  ...           S

In [ ]:
X_test = testData.drop('PRODUCT_ID', axis =1)
y_test = testData['PRODUCT_ID']